In [ ]:
open System.IO

let input = File.ReadAllLines "input.txt"

let testInput = 
    "467..114..
...*......
..35..633.
......#...
617*......
.....+.58.
..592.....
......755.
...$.*....
.664.598..".Split "\n"

let engineText = input

In [ ]:
let rec getNumber substring acc =
    match substring with
    | [] -> acc |> List.rev
    | x::xs -> 
        match x with
        | n when n >= '0' && n <= '9' -> getNumber xs (n :: acc)
        | _ -> acc |> List.rev

In [ ]:
type Engine = 
    | Symbol of char * (int * int)
    | Number of int * ((int*int) list)


let rec parseRow (rowIdx: int) (colIdx: int) (line: char list) acc =
    if line.Length = colIdx then
        acc 
    else
        match line.[colIdx] with
        | '.' -> parseRow rowIdx (colIdx + 1) line acc
        | n when n >= '0' && n <= '9' -> 
            let numberCharacters = getNumber (line.[colIdx..]) []
            let number = numberCharacters |> List.map string |> String.concat "" |> int
            let numberCoordinates = [ for i in 0..numberCharacters.Length-1 -> rowIdx, colIdx + i ]
            parseRow rowIdx (colIdx + numberCharacters.Length) line ((Number (number,numberCoordinates))::acc)
        | symbol -> parseRow rowIdx (colIdx + 1) line ((Symbol (symbol, (rowIdx, colIdx)))::acc)

// Engine specification is an array of numbers or symbols with their coordinates
let engine = 
    engineText
    |> Array.mapi (fun rowIdx row -> 
        parseRow rowIdx 0 (row.ToCharArray() |> List.ofArray) []
        |> Array.ofList)
    |> Array.collect id

engine

In [ ]:
let symbolLocations = 
    engine 
    |> Array.choose (fun x -> 
        match x with 
        | Symbol (_, coords) -> Some coords
        | _ -> None)

symbolLocations

In [33]:
let nextTo (x1,x2) (y1, y2) =  ((abs (x1 - y1)) <= 1) && ((abs (x2 - y2)) <= 1)

let rec isSymbolAdjacent coords =
    match coords with
    | [] -> false
    | x::xs ->
        let adjacentSymbols =
            symbolLocations
            |> Array.filter (fun y -> nextTo x y)
        if adjacentSymbols.Length > 0 then 
            true
        else 
            isAdjacent xs

let part1 =
    engine
    |> Array.choose (fun x ->
        match x with
        | Number (n, coords) -> 
            if isSymbolAdjacent coords then
                Some n
            else 
                None
        | _ -> None)
    |> Array.sum

part1 

[ 527, 307, 773, 845, 453, 998, 65, 861, 331, 164, 105, 196, 752, 942, 377, 74, 516, 839, 418, 745 ... (610 more) ]

In [ ]:
// A gear is any * symbol that is adjacent to exactly two part numbers.
// Its gear ratio is the result of multiplying those two numbers together

let part2 =
    engine
    |> Array.choose (fun x ->
        match x with
        | Symbol ('*', coords) -> Some coords
        | _ -> None)
    |> Array.map (fun symbolCoords -> 
        engine
        |> Array.choose (fun x ->
            match x with
            | Number (n, numberCoords) -> 
                let isNextTo = 
                    numberCoords
                    |> List.fold (fun state nc -> 
                            nextTo nc symbolCoords || state) false
                if isNextTo then Some(n) else None
            | _ -> None)
        |> fun a -> if a.Length = 2 then a.[0] * a.[1] else 0
        )
    |> Array.sum

part2
